In [10]:
!docker stop ipfs_host

ERROR: The Compose file './docker-compose.yml' is invalid because:
Unsupported config option for services.networks: 'backend'


In [11]:
!docker rm ipf_host

ERROR: The Compose file './docker-compose.yml' is invalid because:
Unsupported config option for services.networks: 'backend'


In [12]:
!docker run -d --name ipfs_host2 -v $ipfs_staging:/export -v $ipfs_data:/data/ipfs -p 4001:4001 -p 4001:4001/udp -p 127.0.0.1:8080:8080 -p 127.0.0.1:5001:5001 ipfs/go-ipfs:latest

539f3618e5ff0b0a85e7d61b47c4aeeb89876547f596fe3bad72779d35980a25


In [1]:
!mkdir /tmp/posts
!mkdir /tmp/comments

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException
from substrate_helpers import make_call, addSchema, get_msa_id, \
            get_signature, create_msa_with_delegator, mint_votes, mint_user, get_schemas_from_pattern, \
            get_content_from_schemas, get_user, follow_user, add_delegate

/home/chia/miniconda3/lib/python3.8/site-packages/ipfshttpclient/client/__init__.py:73: VersionMismatch: Unsupported daemon version '0.13.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


In [23]:
bob = Keypair.create_from_uri('//Bob')
substrate = SubstrateInterface(
    url="ws://127.0.0.1:9944",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [42]:
substrate.get_block(substrate.get_block_hash())['extrinsics']

[<GenericExtrinsic(value={'extrinsic_hash': None, 'extrinsic_length': 2539, 'call': {'call_index': '0x0100', 'call_function': 'set_validation_data', 'call_module': 'ParachainSystem', 'call_args': [{'name': 'data', 'type': 'ParachainInherentData', 'value': {'validation_data': {'parent_head': '0x2c47f5c616414b298cb5631145fbce8db6fa1f878898f71e6f18695b87a3b5e1049ce0f96de5e05c1b2bcbae51d945ced7ea85105b1bfded150417f401e4055b4a48340f90aa3ad867daed1d1a5279449b4555d8a076d6517d43214ee13e8a27340806617572612082bd39080000000005617572610101f4d7b63b71ed909dcc5451daef0f92ac9faf2009a1cf2d1c0ed7c68aac324816e0412bd70734935e6c0375c63404222838c1ba9832d38b85d5ed8ed99b7afc81', 'relay_parent_number': 116, 'relay_parent_storage_root': '0x263e8eda56c5a5b91d59d682fd33222bc2ecc7514bc473d530c72f63ac9a9211', 'max_pov_size': 5242880}, 'relay_chain_state': {'trie_nodes': ['0x5f04b49d95320d9021994c850f25b8e3851d03000030000080000008000000000010000000100005000000050000000200000002000000000050000000100000e87648170000000

In [24]:
bob_msa_id = get_msa_id(bob, create=True)

In [25]:
bob_msa_id

1

# Make schemas

In [57]:
post_schema = "category STRING,title STRING,body STRING,url STRING,is_nsfw NUMERIC"
post_schemaId = addSchema(post_schema, wait_for_inclusion=True, wait_for_finalization=False)

comment_schema = "post_hash STRING,parent_hash STRING,depth NUMERIC,body STRING"
comment_schemaId = addSchema(comment_schema, wait_for_inclusion=True, wait_for_finalization=False)

vote_schema = "post_hash STRING,parent_hash STRING,parent_type STRING,num_votes NUMERIC"
vote_schemaId = addSchema(vote_schema, wait_for_inclusion=True, wait_for_finalization=False)

user_schema = "msa_id NUMERIC,username STRING,password STRING,profile_pic STRING,wallet_ss58_address STRING"
user_schemaId = addSchema(user_schema, wait_for_inclusion=True, wait_for_finalization=False)

follow_schema = "protagonist_msa_id NUMERIC,antagonist_msa_id NUMERIC,event STRING"
follow_schemaId = addSchema(follow_schema, wait_for_inclusion=True, wait_for_finalization=False)

link_schema = "account_type STRING,account_value STRING,event STRING"
link_schemaId = addSchema(link_schema, wait_for_inclusion=True, wait_for_finalization=False)

In [58]:
[post_schemaId, comment_schemaId, vote_schemaId, user_schemaId, follow_schemaId, link_schemaId]

[7, 8, 3, 4, 5, 10]

# Mint accounts 

In [62]:
msa_keys = {}
for account in ['Alice', 'Charlie', 'Dave', 'Eve', 'Ferdie']:
    wallet = Keypair.create_from_uri(f'//{account}password')
    user_msa_id = create_msa_with_delegator(bob, wallet)
    receipt_user = mint_user(user_msa_id, account, "password", "profile_pic", wallet)
    msa_keys[account] = user_msa_id

In [63]:
msa_keys

{'Alice': 693, 'Charlie': 382, 'Dave': 694, 'Eve': 695, 'Ferdie': 696}

# Have accounts follow each other

In [64]:
for name1, k1 in msa_keys.items():
    for name2, k2 in msa_keys.items():
        if k1 != k2:
            print(name1, name2)
            receipt = follow_user(k1, k2, True, wait_for_inclusion=False, wait_for_finalization=False)

Alice Charlie
Alice Dave
Alice Eve
Alice Ferdie
Charlie Alice
Charlie Dave
Charlie Eve
Charlie Ferdie
Dave Alice
Dave Charlie
Dave Eve
Dave Ferdie
Eve Alice
Eve Charlie
Eve Dave
Eve Ferdie
Ferdie Alice
Ferdie Charlie
Ferdie Dave
Ferdie Eve


In [65]:
receipt = follow_user(msa_keys['Dave'], msa_keys['Eve'], False, wait_for_inclusion=False, wait_for_finalization=False)
receipt = follow_user(msa_keys['Eve'], msa_keys['Dave'], False, wait_for_inclusion=False, wait_for_finalization=False)
receipt = follow_user(msa_keys['Dave'], msa_keys['Eve'], True, wait_for_inclusion=False, wait_for_finalization=False)

# Test post

In [31]:
%%bash
curl -X 'POST' \
  'http://67.160.97.38:5000/submit/post' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "category": "string",
  "title": "string",
  "body": "string",
  "url": "http://www.google.com",
  "is_nsfw": true
}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:02:09 --:--:--     0
curl: (28) Failed to connect to 67.160.97.38 port 5000: Connection timed out


CalledProcessError: Command 'b'curl -X \'POST\' \\\n  \'http://67.160.97.38:5000/submit/post\' \\\n  -H \'accept: application/json\' \\\n  -H \'Content-Type: application/json\' \\\n  -d \'{\n  "category": "string",\n  "title": "string",\n  "body": "string",\n  "url": "http://www.google.com",\n  "is_nsfw": true\n}\'\n'' returned non-zero exit status 28.